In [3]:
from pathlib import Path
import pandas as pd
import boto3
from io import StringIO




In [8]:
client = boto3.client('s3')
bucket_name = 'columbia-project-resources'
object_key = 'bitcoin.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

bitcoin_df = pd.read_csv(StringIO(csv_string))
bitcoin_df.head()

NoCredentialsError: Unable to locate credentials

#module 11 - Evelyn

In [4]:
import pandas as pd

In [3]:
import datetime as dt

In [2]:
from pathlib import Path

In [2]:
from prophet.forecaster import Prophet



In [4]:
import matplotlib.pyplot as plt

In [5]:
get_ipython().run_line_magic('matplotlib', 'inlineimport')



KeyError: 'inlineimport'

In [6]:
import boto3

In [7]:
from io import StringIO